In [4]:

import os
import pandas as pd
import matplotlib.pyplot as plt
import utils.img_fun as img_fun
import rasterio
import numpy as np
from rasterio.transform import from_origin
from rasterio.windows import Window



### Cargamos la imagen y obtenemos las variables clave (ancho, alto, esquinas)

Rasterio es la liberería que permitirá que las imágenes conserven los metadatos y que estén georeferenciadas aún después de procesarlas y recortarlas. 

In [5]:

# path  = '/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'
# image_name = "orthomosaic_all_big.tif"

NUM_TILE = 22

if os.path.exists('/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'):
    path  = '/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'
else:
    path = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos'

image_name = f"recortes/recorte_{NUM_TILE}.tif"
tiff_file = os.path.join(path, image_name)


# Sacamos un diccionario con toda la información de la imagen
img_info = img_fun.get_img_info(tiff_file)

WIDTH = img_info["width"]
HEIGHT = img_info["height"]
TOP_LEFT = img_info["top_left"]
BOTTOM_RIGHT = img_info["bottom_right"]
min_x, max_y = img_info['top_left']
max_x, min_y = img_info['bottom_right']

Metadatos de la imagen:
driver: GTiff
dtype: uint8
nodata: None
width: 10195
height: 11420
count: 4
crs: EPSG:4326
transform: | 0.00, 0.00,-59.24|
| 0.00,-0.00,-62.29|
| 0.00, 0.00, 1.00|

Información adicional:
Ancho y alto de la imagen: 10195  x  11420
Sistema de coordenadas: EPSG:4326
Transformación: | 0.00, 0.00,-59.24|
| 0.00,-0.00,-62.29|
| 0.00, 0.00, 1.00|
Coordenadas de las esquinas de la imagen:
TOP LEFT: (-59.236389267356785, -62.290013442834734)
BOTTOM RIGHT: (-59.231134009926784, -62.29275364899473)


## Recorte de los tiles y carpeta de salida

1. Se define en cuánto se quiere recortar (en nuestro caso 20 filas y 20 columnas). 
2. Se descartan todos los tiles que tengan una alta proporción de píxeles blanco o negros (mnás del 50%)
3. Acto seguido se procede al recorte y se envía a la carpeta de OUPUT_DIR.

In [6]:

#Directorio de salida
OUTPUT_DIR = os.path.join('cut_tiles', f'tiles_500x500_{NUM_TILE}')
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory: {OUTPUT_DIR}")

# Definición del recorte
ROWS, COLS = 20, 20
tile_width = WIDTH // COLS
tile_height = HEIGHT // ROWS

# Abrir el TIFF original con rasterio
with rasterio.open(tiff_file) as src:
    for i in range(ROWS):
        print("Fila nº ", i)
        for j in range(COLS):
            left = j * tile_width
            upper = i * tile_height
            
            # Definir ventana de recorte
            window = Window(left, upper, tile_width, tile_height)
            
            # Leer el recorte
            cropped_image = src.read(window=window)

            # Identificar píxeles negros (000000)
            is_black = np.all(cropped_image == 0, axis=0)
            
            # Identificar píxeles blancos (FFFFFF)
            is_white = np.all(cropped_image == 255, axis=0)
            
            # Identificar píxeles vacíos o transparentes (suponiendo valor 0 en el canal de alfa o nodata)
            if "nodata" in src.profile:
                nodata_value = src.profile["nodata"]
                is_empty = np.any(cropped_image == nodata_value, axis=0)
            else:
                # Si hay un canal alfa, considera valores 0 como transparentes
                is_empty = cropped_image.shape[0] > 3 and np.all(cropped_image[-1] == 0, axis=0)

            # Calcular proporciones de píxeles negros, blancos y vacíos
            num_pixels = cropped_image.shape[1] * cropped_image.shape[2]  # Total de píxeles
            black_ratio = np.sum(is_black) / num_pixels
            white_ratio = np.sum(is_white) / num_pixels
            empty_ratio = np.sum(is_empty) / num_pixels

            # Guardar solo si todos los umbrales se cumplen
            if black_ratio <= 0.5 and white_ratio <= 0.1 and empty_ratio <= 0.1:
                # Crear los metadatos del recorte
                cropped_meta = src.meta.copy()
                cropped_meta.update({
                    "height": tile_height,
                    "width": tile_width,
                    "transform": rasterio.windows.transform(window, src.transform)
                })
                
                # Guardar la imagen recortada con georreferenciación
                output_path = f"{OUTPUT_DIR}/tile_{NUM_TILE}_subrecorte_{i * COLS + j + 1}.tiff"
                with rasterio.open(output_path, 'w', **cropped_meta) as dst:
                    dst.write(cropped_image)
                print(f"Guardado recorte {i * COLS + j + 1} en {output_path}")
            else:
                print(f"Recorte {i * COLS + j + 1} descartado por exceso de píxeles negros, blancos o vacíos")

    


Output directory: cut_tiles\tiles_500x500_22
Fila nº  0
Recorte 1 descartado por exceso de píxeles negros, blancos o vacíos
Recorte 2 descartado por exceso de píxeles negros, blancos o vacíos
Recorte 3 descartado por exceso de píxeles negros, blancos o vacíos
Recorte 4 descartado por exceso de píxeles negros, blancos o vacíos
Recorte 5 descartado por exceso de píxeles negros, blancos o vacíos
Recorte 6 descartado por exceso de píxeles negros, blancos o vacíos
Recorte 7 descartado por exceso de píxeles negros, blancos o vacíos
Recorte 8 descartado por exceso de píxeles negros, blancos o vacíos
Recorte 9 descartado por exceso de píxeles negros, blancos o vacíos
Guardado recorte 10 en cut_tiles\tiles_500x500_22/tile_22_subrecorte_10.tiff
Guardado recorte 11 en cut_tiles\tiles_500x500_22/tile_22_subrecorte_11.tiff
Guardado recorte 12 en cut_tiles\tiles_500x500_22/tile_22_subrecorte_12.tiff
Guardado recorte 13 en cut_tiles\tiles_500x500_22/tile_22_subrecorte_13.tiff
Guardado recorte 14 en c

### Extracción de las coordenadas de cada tile

Extraemos las coordenadas generales de cada tile y las filtramos.

In [9]:
csv_file = os.path.join(path, 'chinstraps_eca56.csv')
data = pd.read_csv(csv_file, encoding='ISO-8859-1')

print("Datos Originales: \n", data.head())

# Filtrar los puntos que están dentro del tile
filtered_data = data[
    (data['X'] >= min_x) & (data['X'] <= max_x) &
    (data['Y'] >= min_y) & (data['Y'] <= max_y)
]


filtered_data.to_csv(os.path.join('coords', 'coords_per_tile' ,f'coords_{NUM_TILE}.csv'), index=False)
filtered_data.head()

Datos Originales: 
    Id          X          Y
0   0 -59.209791 -62.285558
1   0 -59.209815 -62.285566
2   0 -59.209862 -62.285563
3   0 -59.209878 -62.285563
4   0 -59.209861 -62.285556


,Id,X,Y


### Ejemplo de posibles resultados (para orientación)

*IMAGEN 62*

Esquina superior izquierda: (-59.236389267356785, -62.300974267474736)

Esquina superior derecha: (-59.231134009926784, -62.300974267474736)

Esquina inferior izquierda: (-59.236389267356785, -62.303714473634734)

Esquina inferior derecha: (-59.231134009926784, -62.303714473634734)

---

*IMAGEN 52*

Esquina superior izquierda: (-59.236389267356785, -62.29823406131474)

Esquina superior derecha: (-59.231134009926784, -62.29823406131474)

Esquina inferior izquierda: (-59.236389267356785, -62.300974267474736)

Esquina inferior derecha: (-59.231134009926784, -62.300974267474736)